<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/7-3%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%20%EB%AA%A8%EB%8D%B8%20%ED%9B%88%EB%A0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **7-3 신경망 모델 훈련**



---

작업 중

---



## **손실 곡선**

## **검증 손실**

## **드롭아웃**

## **모델 저장과 복원**

## **콜백**